In [9]:
from pydantic import ValidationError
from strictyaml import load
from boosting_model.config.core import create_and_validate_config

In [5]:
    # Given
    # We make use of the pytest built-in tmpdir fixture





IndentationError: expected an indented block (3351336545.py, line 8)

In [10]:

TEST_CONFIG_TEXT = load(open('trial.yaml','r').read())
create_and_validate_config(parsed_config=TEST_CONFIG_TEXT)

ValidationError: 6 validation errors for AppConfig
pipeline_name
  field required (type=value_error.missing)
training_x_file
  field required (type=value_error.missing)
training_y_file
  field required (type=value_error.missing)
test_x_file
  field required (type=value_error.missing)
test_y_file
  field required (type=value_error.missing)
saved_model_name
  field required (type=value_error.missing)

In [19]:
try:
    create_and_validate_config(parsed_config=TEST_CONFIG_TEXT)

except ValidationError as e:
    f = e
    print(e)

6 validation errors for AppConfig
pipeline_name
  field required (type=value_error.missing)
training_x_file
  field required (type=value_error.missing)
training_y_file
  field required (type=value_error.missing)
test_x_file
  field required (type=value_error.missing)
test_y_file
  field required (type=value_error.missing)
saved_model_name
  field required (type=value_error.missing)


In [17]:
type(f)

pydantic.error_wrappers.ValidationError

In [1]:
from boosting_model import pipeline
from boosting_model.config.core import config
from boosting_model.processing.validation import validate_input
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline
from boosting_model.processing.load_data import load_dataset

In [2]:
x_train = load_dataset(filename=config.app_config.training_x_file)
y_train = load_dataset(filename=config.app_config.training_y_file)

In [3]:
input_data, error = validate_input(input_data=x_train)

/Users/rahul/anacond/anaconda3/lib/python3.9/site-packages/boosting_model/processing/validation.py:37: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  schema.load(input_data.replace(


In [4]:
input_data

,name,online_order,book_table,votes,location,rest_type,cuisines,approx_cost,reviews_list,listed_in_type,listed_in_city
9179,Sri Banashankari Donne Biriyani,1,0,80,Banaswadi,Quick Bites,Biryani,150,"[('Rated 1.0', ""RATED\n Don't ever order in t...",Dine-out,Kammanahalli
25117,RK Fresh Food,1,0,13,BTM,Quick Bites,Fast Food,300,"[('Rated 5.0', 'RATED\n Must try egg grilled ...",Delivery,BTM
10558,Royal Restaurant,0,0,7,Shivajinagar,Quick Bites,"North Indian, Chinese",200,"[('Rated 1.0', 'RATED\n Order was declined af...",Dine-out,Lavelle Road
2972,Namma Kadai,0,0,4,BTM,Quick Bites,South Indian,150,"[('Rated 4.0', 'RATED\n I just explored this ...",Dine-out,Koramangala 7th Block
21445,A1 Garam Masala,1,0,25,Kanakapura Road,Casual Dining,"North Indian, Andhra, Chinese",500,"[('Rated 5.0', 'RATED\n The food was very goo...",Delivery,JP Nagar
...,...,...,...,...,...,...,...,...,...,...,...
5047,Earth Plate Express,1,0,33,JP Nagar,Food Court,"Italian, Continental",600,"[('Rated 4.0', 'RATED\n Well Earth Plate Expr...",Dine-out,JP Nagar
10821,Radha Rani,1,0,244,Brookefield,Quick Bites,"Bengali, North Indian, Chinese",400,"[('Rated 4.0', 'RATED\n One of my favourite b...",Delivery,Brookefield
15497,Chai Street,1,0,5,Koramangala 5th Block,Beverage Shop,Beverages,150,"[('Rated 1.0', 'RATED\n Ordered gulab jamun f...",Delivery,Koramangala 7th Block
16510,Tandoor Garden,1,0,71,BTM,Quick Bites,"North Indian, Chinese, Kebab",350,"[('Rated 5.0', 'RATED\n This place might look...",Delivery,Koramangala 7th Block


In [4]:
config.model_config

ModelConfig(feature_to_drop=['location', 'name'], count_vector=['cuisines'], tfidf=['reviews_list'], categorical=['listed_in_type', 'rest_type', 'listed_in_city'], numerical=['votes', 'approx_cost'], target=['rate'], ordinal=['online_order', 'book_table'], rename={'approx_cost(for two people)': 'approx_cost', 'listed_in(type)': 'listed_in_type', 'listed_in(city)': 'listed_in_city'}, features=['location', 'name', 'cuisines', 'reviews_list', 'listed_in_type', 'rest_type', 'listed_in_city', 'votes', 'approx_cost', 'online_order', 'book_table', 'approx_cost', 'listed_in_type', 'listed_in_city'])

In [5]:
input_data =input_data[config.model_config.features]

In [6]:
  

p = pipeline.pipe
model = pipeline.model_maker(model=XGBRegressor(max_depth=3))

p.fit(input_data)

xx = p.transform(input_data)

model.fit(xx.to_numpy(), y_train)

final_pipe = make_pipeline(p, model)

prediction = final_pipe.predict(input_data)

In [10]:
input_data.columns

Index(['name', 'online_order', 'book_table', 'votes', 'location', 'rest_type',
       'cuisines', 'approx_cost', 'reviews_list', 'listed_in_type',
       'listed_in_city'],
      dtype='object')

In [11]:
x.columns

Index(['cuisines', 'reviews_list', 'listed_in_type', 'rest_type',
       'listed_in_city', 'votes', 'approx_cost', 'online_order', 'book_table',
       'approx_cost', 'listed_in_type', 'listed_in_city'],
      dtype='object')

In [3]:
from boosting_model.processing import preprocessor

In [4]:
assert config.model_config.count_vector[0] in list(x_train.columns)

In [5]:


transformer = preprocessor.sklearnCountVector(
    variable=config.model_config.count_vector)




In [13]:
t = transformer.fit_transform(x_trainj)


NameError: name 'x_trainj' is not defined

In [14]:
assert t is not None

NameError: name 't' is not defined

In [15]:
import typing

In [ ]:
typing.Union